In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch
from model import MaryCausalModel, MaryConfig

/home/gpu/anaconda3/envs/dsy_naive_mllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("./tokenizer")
AutoConfig.register("mary_llm", MaryConfig)
AutoModelForCausalLM.register(MaryConfig, MaryCausalModel)

In [3]:
dpo_model = AutoModelForCausalLM.from_pretrained("save/dpo_1_beyond_trans_chinese")
sft_model = AutoModelForCausalLM.from_pretrained("save/sft")

/home/gpu/anaconda3/envs/dsy_naive_mllm/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
input_data = [tokenizer.bos_token_id] + tokenizer.encode('1+1等于几，并解释为什么等于这个结果')
print(input_data)

[2, 21, 15, 21, 6240, 1920, 272, 583, 2571, 4589, 6240, 992, 2105]


In [11]:
# 分析tokenizer
import collections
import langdetect 
length_dict = collections.defaultdict(int)
for i in range(tokenizer.vocab_size):
    token = tokenizer.decode(i)
    length_dict[i] = len(token)

length_dict = dict(sorted(length_dict.items(), key=lambda x: x[1], reverse=True))  # 按降序
tot = 0
for item in length_dict.items():
    try:
        if langdetect.detect(tokenizer.decode(item)) == 'zh-cn':
            print(tokenizer.decode(item), "encode: "+str(item))
            tot += 1
    except:
        pass
    if tot == 10:
        break

        

以下是一些! encode: (4784, 5)
是一个</s> encode: (1591, 3)
是一种</s> encode: (1930, 3)
我们的</s> encode: (2014, 3)
更好地</s> encode: (2222, 3)
的问题</s> encode: (2583, 3)
过程中</s> encode: (2660, 3)
之间的</s> encode: (2670, 3)
的影响</s> encode: (2690, 3)
可以在</s> encode: (3005, 3)


In [6]:
tokens = dpo_model.generate({"input_ids":torch.tensor(input_data).unsqueeze(0), "labels":None}, tokenizer.eos_token_id, 200)
for token in tokens:
    print(tokenizer.decode(token))

RuntimeError: The size of tensor a (27) must match the size of tensor b (14) at non-singleton dimension 1